In [ ]:
!pip install networkx

In [ ]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 35.3 MB/s eta 0:00:00


In [ ]:
import pennylane as qml
from pennylane import qaoa
import networkx as nx

In [44]:
dev = qml.device('default.qubit', wires=len(g.nodes))

def create_graph_from_friendships(friendships):
    """Create a NetworkX graph from a list of friendships."""
    g = nx.Graph()
    g.add_edges_from(friendships)
    return g

def setup_qaoa_circuit(g, p):
    """Set up the QAOA circuit using PennyLane with p layers."""
    cost_h, mixer_h = qaoa.min_vertex_cover(g, constrained=False)

    @qml.qnode(dev)
    def circuit(params):
        for i in range(p):  # Loop through each of the p layers
            qaoa.cost_layer(params[0][i], cost_h)
            qaoa.mixer_layer(params[1][i], mixer_h)
        return qml.expval(cost_h)

    return circuit

def find_minimum_vertex_cover(friendships, p):
    """Use QAOA to find the minimum vertex cover of the graph."""
    g = create_graph_from_friendships(friendships)
    circuit = setup_qaoa_circuit(g, p)
    params = 0.01 * qml.numpy.random.randn(2, p)  # Initialize parameters for 3 layers
    opt = qml.GradientDescentOptimizer(0.1)
    steps = 100

    # Optimization loop
    for i in range(steps):
        params = opt.step(circuit, params)
        if (i + 1) % 20 == 0:
            print("Cost after step {:5d}: {:.7f}\n".format(i + 1, circuit(params)))

    # Get the final probabilities
    @qml.qnode(dev)
    def probability_circuit(params):
      setup_qaoa_circuit(g, p)
      return qml.probs(wires=range(len(g.nodes)))

    return params

# Assuming 0=Mario, 1=Sarah, 2=Raúl, 3=Ana, 4=Enrique, 5=Saúl
friendships = [(0, 1), (0, 2), (0, 3), (4, 1), (4, 2), (5, 3)]
params = find_minimum_vertex_cover(friendships, p=3)

print("Optimized QAOA parameters for 3 layers:\n", params, "\n")

# Print the probabilities of each node being in the dominating set
probs = probability_circuit(params)
for idx in range(len(g.nodes)):
    print("Probability of node {} being in the dominating set: {:.2f}".format(idx, probs[idx]))

Cost after step    20: -0.7271804

Cost after step    40: -0.3915114

Cost after step    60: -0.5793345

Cost after step    80: -0.5523641

Cost after step   100: -1.3323962

Optimized QAOA parameters for 3 layers:
 [[-0.00417122 -0.97163713 -0.70588191]
 [-0.54161311  0.47074677  1.41704356]] 

Probability of node 0 being in the dominating set: 0.00
Probability of node 1 being in the dominating set: 0.00
Probability of node 2 being in the dominating set: 0.00
Probability of node 3 being in the dominating set: 0.01
Probability of node 4 being in the dominating set: 0.00
Probability of node 5 being in the dominating set: 0.01


In [39]:
#Optmial U(C) and U(B)
cost_h, cc = qaoa.min_vertex_cover(g, constrained=False)